# GEOCODING  

The aim of this project is to geocode some knowns location within Ghana's two main cities which are Accra and Kumasi.


In [1]:
import geopy
import geopandas as gpd
import pandas as pd
from geopy.geocoders import Nominatim

import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
#Read the data 
df = pd.read_csv("Ghana_Project.csv")
df.head()

#Merge the needed Fields
df["ADDRESS"] = df["Name"] +', '+ df["City"] +', ' + df["Country"]
df.head()

,Name,City,Country,ADDRESS
0,Ghana International School,Accra,Ghana,"Ghana International School, Accra, Ghana"
1,High Commission of the Republic of Sierra Leone,Accra,Ghana,High Commission of the Republic of Sierra Leon...
2,University of Ghana,Accra,Ghana,"University of Ghana, Accra, Ghana"
3,37 Military Hospital,Accra,Ghana,"37 Military Hospital, Accra, Ghana"
4,Accra International Conference Center,Accra,Ghana,"Accra International Conference Center, Accra, ..."


Import rate limiter to delay the geocoding calls between the addresses

In [3]:
geolocator = Nominatim(user_agent="John")
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode, min_delay_seconds =1)

Location column must be created to store the coordinates as a tuple (Actual geocoding)

In [4]:
#df['location'] = df['ADDRESS'].apply(geocode)
df.head(6)

,Name,City,Country,ADDRESS
0,Ghana International School,Accra,Ghana,"Ghana International School, Accra, Ghana"
1,High Commission of the Republic of Sierra Leone,Accra,Ghana,High Commission of the Republic of Sierra Leon...
2,University of Ghana,Accra,Ghana,"University of Ghana, Accra, Ghana"
3,37 Military Hospital,Accra,Ghana,"37 Military Hospital, Accra, Ghana"
4,Accra International Conference Center,Accra,Ghana,"Accra International Conference Center, Accra, ..."
5,Banquet Hall,Accra,Ghana,"Banquet Hall, Accra, Ghana"


Create longitude, latitude and altitude from location column above

In [15]:
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

KeyError: 'location'

##### split the point columns into three; latitudes, longitudes and altitudes

In [14]:
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

KeyError: 'point'

clean unwanted columns

In [7]:
df = df.drop(['City', 'Country', 'ADDRESS', 'location', 'point'], axis = 1)

KeyError: "['location' 'point'] not found in axis"

display the result

In [8]:
df.head()

,Name,City,Country,ADDRESS
0,Ghana International School,Accra,Ghana,"Ghana International School, Accra, Ghana"
1,High Commission of the Republic of Sierra Leone,Accra,Ghana,High Commission of the Republic of Sierra Leon...
2,University of Ghana,Accra,Ghana,"University of Ghana, Accra, Ghana"
3,37 Military Hospital,Accra,Ghana,"37 Military Hospital, Accra, Ghana"
4,Accra International Conference Center,Accra,Ghana,"Accra International Conference Center, Accra, ..."


Avoid Nan error by dropping the rows that are empty

In [9]:
df=df.dropna(subset=['longitude'])

df=df.dropna(subset=['latitude'])

KeyError: ['longitude']

In [10]:
map1 = folium.Map(
    location = [5.580337, -0.176469],
    tiles = 'Stamen Toner',
zoom_start=7,
)

In [11]:
df.apply(lambda row:folium.Marker(
    location=[row["latitude"],
              row["longitude"]],
    popup = row['Name'], 
    icon=folium.Icon(color='blue')
    ).add_to(map1), axis=1)
map1

KeyError: 'latitude'

In [12]:
# Save the map
map1.save("Ghana_geocoded_Address.html")